In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# tf.config.list_physical_devices('GPU')
# tf.test.is_built_with_cuda()
tf.test.is_gpu_available()
# tf.__version__


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value =32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value =32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu',
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value =32, max_value=128, step=16),
        activation='relu',
    ),
    keras.layers.Dense(10, 'softmax')
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
      loss = keras.losses.sparse_categorical_crossentropy,
      # loss='sparse_categorical_crossentropy', 
      metrics=['accuracy']
  )

  return model

In [10]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
# The Output folder must be deleted
tuner = RandomSearch(
    build_model, 
    objective='val_accuracy', 
    max_trials=5, 
    directory='output', 
    project_name='Mnist Fashion'
    )

INFO:tensorflow:Reloading Oracle from existing project output\Mnist Fashion\oracle.json


In [12]:
tuner.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 01m 25s]
val_accuracy: 0.8611666560173035

Best val_accuracy So Far: 0.9085000157356262
Total elapsed time: 00h 06m 21s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        134448    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2230368   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,366,906
Trainable params: 2,366,906
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 29s 17ms/step - loss: 0.1367 - accuracy: 0.9493 - val_loss: 0.2774 - val_accuracy: 0.9042
Epoch 5/10
1688/1688 [==============================] - 26s 15ms/step - loss: 0.0947 - accuracy: 0.9650 - val_loss: 0.2687 - val_accuracy: 0.9122
Epoch 6/10
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0617 - accuracy: 0.9771 - val_loss: 0.3537 - val_accuracy: 0.9082
Epoch 7/10
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0427 - accuracy: 0.9839 - val_loss: 0.3818 - val_accuracy: 0.9138
Epoch 8/10
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0350 - accuracy: 0.9877 - val_loss: 0.4830 - val_accuracy: 0.9078
Epoch 9/10
1688/1688 [==============================] - 28s 16ms/step - loss: 0.0271 - accuracy: 0.9908 - val_loss: 0.4709 - val_accuracy: 0.9088
Epoch 10/10
1688/1688 [==============================] - 28s 16ms/step - loss: 0.0218 - accuracy: 0.9928 - val_loss: 0.5132 

In [16]:
model

In [ ]:
model.summary()